# Stanford CME 241 (Winter 2021) - Assignment 3

#### 4  MDP Bellman Policy Equations for a Deterministric Policy 

Assuming a deterministric policy $\pi_D: S \rightarrow A$, for all $s \in N$:
<br> $$V^{\pi_D} (s) = R^{\pi_D} (s) + \gamma \displaystyle\sum_{s' \in N} P^{\pi_D} (s, s') V^{\pi_D} (s') $$ <div style="text-align: right"> (1) </div>
<br> $$V^{\pi_D} (s) = Q^{\pi_D} (s, a) $$ <div style="text-align: right"> (2) </div>
<br> $$Q^{\pi_D} (s, a) = R(s,a) + \gamma \displaystyle\sum_{s' \in N} P(s, a, s') V^{\pi_D} (s') $$ <div style="text-align: right"> (3) </div>
<br> Considering (2) and (3),
<br> $$Q^{\pi_D} (s, a) = R(s,a) + \gamma \displaystyle\sum_{s' \in N} P(s, a, s') Q^{\pi_D} (s', a)$$ <div style="text-align: right"> (4) </div>

#### Application of MDP Bellman Optimality Equation

* Optimal Value Function

With reference to the MDP State-Value Function Bellman Optimality Equation, $V^{*} (s) = max_{a \in A} \{R(s, a) + \gamma \displaystyle\sum_{s' \in N} P(s, a, s') V^{*}(s')\}$
<br> For all $s \in S $, the optimial value function in this question can be derived as: 
<br> $$V^{*} (s) = max_{a \in [0,1]} \{a \times [(1-a) + 0.5 V^{*} (s')] + (1-a) \times [(1+a) + 0.5 V^{*} (s')] \} $$ 
<br> Since each state in the question has the same values for transition probability and reward function, we can use $V^{*}$ to directly substitute for $V^{*} (s)$ and $V^{*} (s')$.
<br> Thus, $$ V^{*} = max_{a \in [0,1]} \{a \times [(1-a) + 0.5 V^{*}] + (1-a) \times [(1+a) + 0.5 V^{*}] \} $$
<br> $\implies$ $$ V^{*} = max_{a \in [0,1]} \{-2a^{2} +a + 1\} $$

* Optimal Deterministic Policy

With $a \in [0,1]$, the value function is maximized ($V^{*} = 1 $) when $\pi ^{*} (s) = a = 0$ for all $s \in S$.

#### Frog-escape Problem

* State space: $$S = \{i | 0 \leq i \leq n\}$$

* Action space: $$ A = \{ A, B\}$$

* Transitions function

For $1 \leq i \leq n-1 $:
<br> $$P [i'| ( i, A)] = \begin{gather*}
\begin{cases}
  \frac{i}{n}  & i' = i - 1\\    
  \frac{n-1}{n}  & i' = i + 1\\  
  0  & otherwise
\end{cases}
\end{gather*}$$
<br>  $$P [i'| ( i, B)] = \begin{gather*}
\begin{cases}
  \frac{1}{n}  & 0 \leq i' \leq n, i' \neq i \\    
  0  & i' = i
\end{cases}
\end{gather*}$$

* Rewards function

For $1 \leq i \leq n-1$ and $a \in \{ A, B\}$:
<br> $$R [i'| ( i, a)] = \begin{gather*}
\begin{cases}
  1  & i' = n \\    
  0  & otherwise
\end{cases}
\end{gather*}$$

*  MDP

In [14]:
from typing import Mapping
import numpy as np

In [15]:
def lilypads_mdp(n: int):
    mdp = {0: {'crock_A': {0: (1., 0.)}, 'crock_B': {0: (1., 0.)}}}
    for i in range(1,n):
        mdp_i = {i: {'crock_A': {i-1: (i/n, 0.), i+1: (1.-i/n,1. if i == n-1 else 0.)},\
                    'crock_B': {j: (1/n, 1. if j == n else 0.) for j in range(n+1) if j != i}}}
        mdp.update(mdp_i)
    mdp_n = {n: {'crock_A': {n: (1., 0.)}, 'crock_B': {n: (1., 0.)}}}
    mdp.update(mdp_n)
    return mdp

In [16]:
def lilypads_finite_policy(n: int) -> Mapping[int, float]:
    value_function = [0.5] * (n+1)
    value_function[0] = 0.
    value_function[n] = 0.
    tolerance = 1e-8
    epsilon = tolerance * 1e4
    while epsilon >= tolerance:
        old_value_function = [v for v in value_function]
        for i in range(1, n):
            value_function[i] = max((1. if i == n-1 else 0.) + i * value_function[i-1] + (n-1) * value_function[i+1],\
                                    1.+sum(value_function[j] for j in range(1,n) if j != i))/n
        epsilon = max(abs(old_value_function[i]-v) for i,v in enumerate(value_function))
    return {v:f for v,f in enumerate(value_function)}

#### MDP with Normal Distribution Transition (γ = 0)

The optimal cost in this question can be expressed as the following optimal value function:
<br> $$ V^{*} (s) = max_{a \in \mathbb{R}} \{ \mathbb{E}_{s'\sim N(s, \sigma^{2})} (-e^{as'})\}$$
<br> $$ = max_{a \in \mathbb{R}} \{ \int\limits_{-\infty}^{+\infty} \frac{-1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-(s+a\sigma^2))^2}{2\sigma_2}} e^{sa + \frac{\sigma^2 a^2}{2}}dx\}$$
<br> $$ = max_{a \in \mathbb{R}} \{-e^{sa + \frac{\sigma^2 a^2}{2}} \mathbb{E}_{x \sim N(s+a\sigma^2, \sigma^2)} 1\} $$
<br>$$ = max_{a \in \mathbb{R}} \{-e^{sa + \frac{\sigma^2 a^2}{2}} \}$$
<br>The value function is maximized when $$ \frac{\mathrm d}{\mathrm d a} \big( -e^{sa + \frac{\sigma^2 a^2}{2}} \big) = 0$$
<br> $$a=\frac{-s}{\sigma^2}$$
<br> In this case: $$V^{*} (s)= - e^{-\frac{s^2}{2\sigma^2}}$$